In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !python train.py --phase train or test --dataset_path .../mvtec_anomaly_detection --category carpet --project_root_path path/to/save/results --coreset_sampling_ratio 0.01 --n_neighbors 9

## 训练
参数默认：
```python
python train.py --phase train --dataset_path /root/dataset/public/Research/DataSet/Anomaly_Detect/MVTec_AD/mvtec_anomaly_detection/ --category carpet --project_root_path /root/project/wood/ort/trt/research/wdcv/demo/anomaly_detection/models/patchcore
``` 
carpet {'img_auc': 0.930577849117175, 'pixel_auc': 0.93455954390873}

bottle {'img_auc': 0.35952380952380947, 'pixel_auc': 0.45279845194073653}

grid {'img_auc': 0.6432748538011696, 'pixel_auc': 0.3221241193202348}

In [3]:
# !python train.py --phase train --dataset_path /root/dataset/wood/ort/trt/Research/industry_data/ --category bottle --project_root_path /root/project/wood/ort/trt/research/wdcv/demo/anomaly_detection/models/patchcore

#  python train.py --phase train --dataset_path /root/dataset/public/Research/DataSet/Anomaly_Detect/MVTec_AD/mvtec_anomaly_detection/ --category carpet --project_root_path /root/project/wood/ort/trt/research/wdcv/demo/anomaly_detection/models/patchcore --coreset_sampling_ratio 0.01

In [5]:
# !python train.py --phase train --dataset_path /root/dataset/public/Research/DataSet/Anomaly_Detect/MVTec_AD/mvtec_anomaly_detection/ --category bottle --project_root_path /root/project/wood/ort/trt/research/wdcv/demo/anomaly_detection/models/patchcore

In [6]:
!python train_wood.py

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
## 1、model ##
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
/root/conda/envs/ts_env/lib/python3.7/site-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
## 2、train ##
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
/root/conda/envs/ts_env/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  warnings.warn(*args, **kwargs)

  | Name          | Type      | Params
--------------------------------------------
0 | model         | ResNet    | 68.9 M
1 | criterion     | MSELoss   | 0     
2 | inv_normalize | Normalize | 0     
--------------------------------------------
0    

## 测试

In [7]:
# !python train.py --phase test --dataset_path /root/dataset/public/Research/DataSet/Anomaly_Detect/MVTec_AD/mvtec_anomaly_detection/ --project_root_path /root/project/wood/ort/trt/research/wdcv/demo/anomaly_detection/models/patchcore

In [8]:
!python train_wood.py --phase test

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
## 1、model ##
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
/root/conda/envs/ts_env/lib/python3.7/site-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
#### test_dataloader
### defect_types: ['broken_large', 'good', 'broken_small', 'contamination']
/root/conda/envs/ts_env/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
Testing: 

## konwleadges

In [1]:
import torch
import torch.nn as nn

# 创建模型的输出和目标值
predictions = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
targets = torch.tensor([2.0, 2.5, 3.5])

# 定义均方误差损失函数 ==>度量模型的预测值与实际目标值之间的平方差的平均值。
criterion = nn.MSELoss()

# 计算均方误差
loss = criterion(predictions, targets)

# 打印损失值
print("Mean Squared Error:", loss.item())

# 反向传播和优化步骤（在训练阶段使用）
loss.backward()
# 进行优化步骤...


Mean Squared Error: 0.5


In [5]:
1/3 * ((1-2)*(1-2) + (2-2.5)*(2-2.5) + (3-3.5)*(3-3.5))

0.5

In [6]:
import torch
import torch.nn.functional as F

def embedding_concat(x, y):
    B, C1, H1, W1 = x.size()
    _, C2, H2, W2 = y.size()
    s = int(H1 / H2)

    # 使用unfold函数对x进行操作
    x = F.unfold(x, kernel_size=s, dilation=1, stride=s)
    x = x.view(B, C1, -1, H2, W2)

    # 创建一个全零的张量z，形状为(B, C1 + C2, x.size(2), H2, W2)
    z = torch.zeros(B, C1 + C2, x.size(2), H2, W2)

    # 对x的每个切片进行连接，连接的方式是在特征维度上进行拼接
    for i in range(x.size(2)):
        z[:, :, i, :, :] = torch.cat((x[:, :, i, :, :], y), 1)

    # 重塑z的形状，将其形状变为(B, -1, H2 * W2)
    z = z.view(B, -1, H2 * W2)

    # 使用fold函数对z进行操作，将其还原成原始图像的形状
    z = F.fold(z, kernel_size=s, output_size=(H1, W1), stride=s)

    # 返回连接后的张量z
    return z

# 创建两个简单的张量
x = torch.randn(1, 3, 6, 6)  # 1个样本，3个通道，6x6大小的图像
y = torch.randn(1, 2, 3, 3)  # 1个样本，2个通道，3x3大小的图像

# 使用embedding_concat连接这两个张量
result = embedding_concat(x, y)

# 打印结果形状
print(result.shape)


torch.Size([1, 5, 6, 6])


In [7]:
import torch

def reshape_embedding(embedding):
    embedding_list = []
    for k in range(embedding.shape[0]):  # 样本维度
        for i in range(embedding.shape[1]):  # 特征维度
            for j in range(embedding.shape[2]):  # 高度维度
                for l in range(embedding.shape[3]):  # 宽度维度
                    embedding_list.append(embedding[k, i, j, l])
    return embedding_list

# 创建一个简单的二维张量，表示一个样本，每个像素有三个特征
embedding = torch.randn(1, 3, 2, 2)

# 使用 reshape_embedding 函数展平张量
embedding_list = reshape_embedding(embedding)

# 打印结果
print(embedding_list)


[tensor(1.3598), tensor(2.2120), tensor(0.7353), tensor(1.7462), tensor(0.0413), tensor(0.0965), tensor(0.7379), tensor(-0.0076), tensor(-0.0660), tensor(0.1063), tensor(0.8644), tensor(0.0288)]


In [9]:
len(embedding_list)

12